Bubble diagram / dynamic relaxation



In [7]:
import pandas as pd
import numpy as np

In [11]:
#inputs

#number of specific function
num_shop = 4
num_square = 2
num_workshop = num_shop // 2
num_storage = num_shop

num_functions = num_shop + num_square + num_workshop + num_storage
functions = ["shop"] * num_shop + ["square"] * num_square + ["workshop"] * num_workshop + ["storage"] * num_storage
print("Total number of functions:",num_functions)

#m2
area = {
"shop" : 30,
"square" : 2500,
"workshop" : 60,
"storage" : 10
}

Total number of functions: 12


In [12]:
#make a table with all the data regardinf the functions

last_function = ""
i = 0
function_data = []
for func in functions:
    if last_function != func:
        i = 0
    function_data.append([func, i, area[func]])
    last_function = func
    i += 1

pd.DataFrame(function_data, columns=["function_name", "space_index", "area"])

,function_name,space_index,area
0,shop,0,30
1,shop,1,30
2,shop,2,30
3,shop,3,30
4,square,0,2500
5,square,1,2500
6,workshop,0,60
7,workshop,1,60
8,storage,0,10
9,storage,1,10


In [13]:
#make connections
G_edges = [ (0, 1),
            (0, 3),
            (1, 2),
            (1, 3),
            (2, 3),
            (2, 4),
            (0, 4)]

G = nx.Graph()
G.add_nodes_from(G_nodes)
G.add_edges_from(G_edges)
#connectivity matrix met getallen tussen 0 en 1 die de kracht van de veer bepalen
# VV is a matrix with this shape: number of nodes by number of nodes
VV_shape = (num_functions, num_functions)
# Initialize the empty VV
VV = np.zeros(VV_shape, dtype=int)

# display as pandas dataframe
display(pd.DataFrame(VV))

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#place points
pos = { 
    0: (5, 0.03),
    1: (0, 0), 
    2: (-1, 0.3), 
    3: (2, 0.17), 
    4: (4, 0.255)}

In [ ]:
#give the points volumes

#per functie een cirkel maken en die plaatsen op de punten zodat je volumes er aan toe wijst

In [ ]:
#give an orientation to the site

#

In [ ]:
#plot the graph
options = {
    "font_size": 30,
    "node_size": 1500,
    "node_color": "white",
    "edgecolors": "grey",
    "edge_color": "black",
    "linewidths": 3,
    "width": 3,
}

nx.draw_networkx(G, pos, **options)